In [1]:
import sys
sys.path.append("../classes")

In [2]:
import pickle
from sklearn.svm import SVC

from sklearn.feature_selection import SelectKBest
from sklearn.decomposition import PCA

import numpy as np, pandas as pd
from geno_classifier import *
from geno_utils import parse_database
from itertools import starmap

import GEOparse

In [3]:
colon_cancer_dataset = GEOparse.get_GEO(filepath='../GDS3268_full.soft').table

22-Jan-2018 20:57:41 INFO GEOparse - Parsing ../GDS3268_full.soft: 
22-Jan-2018 20:57:41 DEBUG GEOparse - DATABASE: Geo
22-Jan-2018 20:57:41 DEBUG GEOparse - DATASET: GDS3268
22-Jan-2018 20:57:41 DEBUG GEOparse - SUBSET: GDS3268_1
22-Jan-2018 20:57:41 DEBUG GEOparse - SUBSET: GDS3268_2
22-Jan-2018 20:57:41 DEBUG GEOparse - SUBSET: GDS3268_3
22-Jan-2018 20:57:41 DEBUG GEOparse - SUBSET: GDS3268_4
22-Jan-2018 20:57:41 DEBUG GEOparse - SUBSET: GDS3268_5
22-Jan-2018 20:57:41 DEBUG GEOparse - SUBSET: GDS3268_6
22-Jan-2018 20:57:41 DEBUG GEOparse - SUBSET: GDS3268_7
22-Jan-2018 20:57:41 DEBUG GEOparse - SUBSET: GDS3268_8
22-Jan-2018 20:57:41 DEBUG GEOparse - ANNOTATION: 
22-Jan-2018 20:57:41 ERROR GEOparse - Cannot recognize type Annotation
22-Jan-2018 20:57:41 DEBUG GEOparse - DATASET: GDS3268
/home/ammar/virtualenv/bio/lib/python3.5/site-packages/GEOparse/GEOparse.py:330 FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are dif

In [4]:
healthy_raw = """
#GSM282855 = Value for GSM282855: 5192 Normal Uninflamed sigmoid colon; src: Normal Uninflamed sigmoid colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282856 = Value for GSM282856: 5193 Normal Uninflamed sigmoid colon; src: Normal Uninflamed sigmoid colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282857 = Value for GSM282857: 5194 Normal Uninflamed sigmoid colon; src: Normal Uninflamed sigmoid colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282858 = Value for GSM282858: 5195 Normal Uninflamed sigmoid colon; src: Normal Uninflamed sigmoid colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282859 = Value for GSM282859: 5196 Normal Uninflamed sigmoid colon; src: Normal Uninflamed sigmoid colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282860 = Value for GSM282860: 5197 Normal Uninflamed sigmoid colon; src: Normal Uninflamed sigmoid colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282861 = Value for GSM282861: 5198 Normal Uninflamed sigmoid colon; src: Normal Uninflamed sigmoid colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282862 = Value for GSM282862: 5199 Normal Uninflamed sigmoid colon; src: Normal Uninflamed sigmoid colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282863 = Value for GSM282863: 5200 Normal Uninflamed sigmoid colon; src: Normal Uninflamed sigmoid colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282864 = Value for GSM282864: 5201 Normal Uninflamed sigmoid colon; src: Normal Uninflamed sigmoid colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282865 = Value for GSM282865: 5202 Normal Uninflamed sigmoid colon; src: Normal Uninflamed sigmoid colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282866 = Value for GSM282866: 5203 Normal Uninflamed sigmoid colon; src: Normal Uninflamed sigmoid colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282867 = Value for GSM282867: 5204 Normal Uninflamed sigmoid colon; src: Normal Uninflamed sigmoid colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282868 = Value for GSM282868: 5205 Normal Uninflamed sigmoid colon; src: Normal Uninflamed sigmoid colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282869 = Value for GSM282869: 5206 Normal Uninflamed sigmoid colon; src: Normal Uninflamed sigmoid colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282870 = Value for GSM282870: 5207 Normal Uninflamed sigmoid colon; src: Normal Uninflamed sigmoid colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282871 = Value for GSM282871: 5208 Normal Uninflamed sigmoid colon; src: Normal Uninflamed sigmoid colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282872 = Value for GSM282872: 5209 Normal Uninflamed sigmoid colon; src: Normal Uninflamed sigmoid colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282904 = Value for GSM282904: 11389 Normal Uninflamed sigmoid colon; src: Normal Uninflamed sigmoid colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282910 = Value for GSM282910: 11395 Normal Uninflamed sigmoid colon; src: Normal Uninflamed sigmoid colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282913 = Value for GSM282913: 11398 Normal Uninflamed sigmoid colon; src: Normal Uninflamed sigmoid colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282915 = Value for GSM282915: 11400 Normal Uninflamed sigmoid colon; src: Normal Uninflamed sigmoid colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282921 = Value for GSM282921: 11406 Normal Uninflamed sigmoid colon; src: Normal Uninflamed sigmoid colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282927 = Value for GSM282927: 12462 Normal Uninflamed sigmoid colon; src: Normal Uninflamed sigmoid colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282873 = Value for GSM282873: 5210 Normal Uninflamed terminal ileum; src: Normal Uninflamed terminal ileum; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282874 = Value for GSM282874: 5211 Normal Uninflamed terminal ileum; src: Normal Uninflamed terminal ileum; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282875 = Value for GSM282875: 10609 Normal Uninflamed terminal ileum; src: Normal Uninflamed terminal ileum; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282905 = Value for GSM282905: 11390 Normal Uninflamed terminal ileum; src: Normal Uninflamed terminal ileum; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282914 = Value for GSM282914: 11399 Normal Uninflamed terminal ileum; src: Normal Uninflamed terminal ileum; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282918 = Value for GSM282918: 11403 Normal Uninflamed terminal ileum; src: Normal Uninflamed terminal ileum; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282876 = Value for GSM282876: 11081 Normal Uninflamed descending colon; src: Normal Uninflamed descending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282877 = Value for GSM282877: 11082 Normal Uninflamed descending colon; src: Normal Uninflamed descending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282878 = Value for GSM282878: 11083 Normal Uninflamed descending colon; src: Normal Uninflamed descending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282879 = Value for GSM282879: 11084 Normal Uninflamed descending colon; src: Normal Uninflamed descending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282880 = Value for GSM282880: 11085 Normal Uninflamed descending colon; src: Normal Uninflamed descending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282881 = Value for GSM282881: 11086 Normal Uninflamed descending colon; src: Normal Uninflamed descending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282882 = Value for GSM282882: 11087 Normal Uninflamed descending colon; src: Normal Uninflamed descending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282883 = Value for GSM282883: 11088 Normal Uninflamed descending colon; src: Normal Uninflamed descending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282884 = Value for GSM282884: 11089 Normal Uninflamed descending colon; src: Normal Uninflamed descending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282885 = Value for GSM282885: 11090 Normal Uninflamed descending colon; src: Normal Uninflamed descending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282886 = Value for GSM282886: 11091 Normal Uninflamed descending colon; src: Normal Uninflamed descending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282887 = Value for GSM282887: 11092 Normal Uninflamed descending colon; src: Normal Uninflamed descending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282888 = Value for GSM282888: 11093 Normal Uninflamed descending colon; src: Normal Uninflamed descending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282889 = Value for GSM282889: 11094 Normal Uninflamed descending colon; src: Normal Uninflamed descending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282890 = Value for GSM282890: 11095 Normal Uninflamed descending colon; src: Normal Uninflamed descending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282902 = Value for GSM282902: 11387 Normal Uninflamed descending colon; src: Normal Uninflamed descending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282903 = Value for GSM282903: 11388 Normal Uninflamed descending colon; src: Normal Uninflamed descending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282907 = Value for GSM282907: 11392 Normal Uninflamed descending colon; src: Normal Uninflamed descending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282909 = Value for GSM282909: 11394 Normal Uninflamed descending colon; src: Normal Uninflamed descending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282912 = Value for GSM282912: 11397 Normal Uninflamed descending colon; src: Normal Uninflamed descending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282920 = Value for GSM282920: 11405 Normal Uninflamed descending colon; src: Normal Uninflamed descending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282924 = Value for GSM282924: 12459 Normal Uninflamed descending colon; src: Normal Uninflamed descending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282891 = Value for GSM282891: 11376 Normal Uninflamed ascending colon; src: Normal Uninflamed ascending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282892 = Value for GSM282892: 11377 Normal Uninflamed ascending colon; src: Normal Uninflamed ascending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282893 = Value for GSM282893: 11378 Normal Uninflamed ascending colon; src: Normal Uninflamed ascending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282894 = Value for GSM282894: 11379 Normal Uninflamed ascending colon; src: Normal Uninflamed ascending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282895 = Value for GSM282895: 11380 Normal Uninflamed ascending colon; src: Normal Uninflamed ascending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282896 = Value for GSM282896: 11381 Normal Uninflamed ascending colon; src: Normal Uninflamed ascending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282897 = Value for GSM282897: 11382 Normal Uninflamed ascending colon; src: Normal Uninflamed ascending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282898 = Value for GSM282898: 11383 Normal Uninflamed ascending colon; src: Normal Uninflamed ascending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282899 = Value for GSM282899: 11384 Normal Uninflamed ascending colon; src: Normal Uninflamed ascending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282900 = Value for GSM282900: 11385 Normal Uninflamed ascending colon; src: Normal Uninflamed ascending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282901 = Value for GSM282901: 11386 Normal Uninflamed ascending colon; src: Normal Uninflamed ascending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282906 = Value for GSM282906: 11391 Normal Uninflamed ascending colon; src: Normal Uninflamed ascending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282908 = Value for GSM282908: 11393 Normal Uninflamed ascending colon; src: Normal Uninflamed ascending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282911 = Value for GSM282911: 11396 Normal Uninflamed ascending colon; src: Normal Uninflamed ascending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282916 = Value for GSM282916: 11401 Normal Uninflamed ascending colon; src: Normal Uninflamed ascending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282919 = Value for GSM282919: 11404 Normal Uninflamed ascending colon; src: Normal Uninflamed ascending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282923 = Value for GSM282923: 12458 Normal Uninflamed ascending colon; src: Normal Uninflamed ascending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)"""

In [5]:
unhealthy_raw = """
#GSM282929 = Value for GSM282929: 5213 UC Inflamed sigmoid colon; src: UC Inflamed sigmoid colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282933 = Value for GSM282933: 5217 UC Inflamed sigmoid colon; src: UC Inflamed sigmoid colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282936 = Value for GSM282936: 5222 UC Inflamed sigmoid colon; src: UC Inflamed sigmoid colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282937 = Value for GSM282937: 5223 UC Inflamed sigmoid colon; src: UC Inflamed sigmoid colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282942 = Value for GSM282942: 5240 UC Inflamed sigmoid colon; src: UC Inflamed sigmoid colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282945 = Value for GSM282945: 5243 UC Inflamed sigmoid colon; src: UC Inflamed sigmoid colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282954 = Value for GSM282954: 5252 UC Inflamed sigmoid colon; src: UC Inflamed sigmoid colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282961 = Value for GSM282961: 5259 UC Inflamed sigmoid colon; src: UC Inflamed sigmoid colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282964 = Value for GSM282964: 5262 UC Inflamed sigmoid colon; src: UC Inflamed sigmoid colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282965 = Value for GSM282965: 5263 UC Inflamed sigmoid colon; src: UC Inflamed sigmoid colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282967 = Value for GSM282967: 5265 UC Inflamed sigmoid colon; src: UC Inflamed sigmoid colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282969 = Value for GSM282969: 5267 UC Inflamed sigmoid colon; src: UC Inflamed sigmoid colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282970 = Value for GSM282970: 5268 UC Inflamed sigmoid colon; src: UC Inflamed sigmoid colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282972 = Value for GSM282972: 5270 UC Inflamed sigmoid colon; src: UC Inflamed sigmoid colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282973 = Value for GSM282973: 5271 UC Inflamed sigmoid colon; src: UC Inflamed sigmoid colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282975 = Value for GSM282975: 5273 UC Inflamed sigmoid colon; src: UC Inflamed sigmoid colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282996 = Value for GSM282996: 11408 UC Inflamed sigmoid colon; src: UC Inflamed sigmoid colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282999 = Value for GSM282999: 11411 UC Inflamed sigmoid colon; src: UC Inflamed sigmoid colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM283014 = Value for GSM283014: 11430 UC Inflamed sigmoid colon; src: UC Inflamed sigmoid colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM283019 = Value for GSM283019: 11437 UC Inflamed sigmoid colon; src: UC Inflamed sigmoid colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM283026 = Value for GSM283026: 12464 UC Inflamed sigmoid colon; src: UC Inflamed sigmoid colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM283029 = Value for GSM283029: 12467 UC Inflamed sigmoid colon; src: UC Inflamed sigmoid colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM283030 = Value for GSM283030: 12468 UC Inflamed sigmoid colon; src: UC Inflamed sigmoid colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM283033 = Value for GSM283033: 12471 UC Inflamed sigmoid colon; src: UC Inflamed sigmoid colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM283035 = Value for GSM283035: 12473 UC Inflamed sigmoid colon; src: UC Inflamed sigmoid colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM283036 = Value for GSM283036: 12474 UC Inflamed sigmoid colon; src: UC Inflamed sigmoid colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM283038 = Value for GSM283038: 12476 UC Inflamed sigmoid colon; src: UC Inflamed sigmoid colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM283046 = Value for GSM283046: 12484 UC Inflamed sigmoid colon; src: UC Inflamed sigmoid colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM283050 = Value for GSM283050: 12488 UC Inflamed sigmoid colon; src: UC Inflamed sigmoid colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM283053 = Value for GSM283053: 12491 UC Inflamed sigmoid colon; src: UC Inflamed sigmoid colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM283055 = Value for GSM283055: 12493 UC Inflamed sigmoid colon; src: UC Inflamed sigmoid colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM283056 = Value for GSM283056: 12494 UC Inflamed sigmoid colon; src: UC Inflamed sigmoid colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282928 = Value for GSM282928: 5212 UC Inflamed terminal ileum; src: UC Inflamed terminal ileum; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282930 = Value for GSM282930: 5214 UC Inflamed descending colon; src: UC Inflamed descending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282932 = Value for GSM282932: 5216 UC Inflamed descending colon; src: UC Inflamed descending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282934 = Value for GSM282934: 5218 UC Inflamed descending colon; src: UC Inflamed descending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282976 = Value for GSM282976: 5274 UC Inflamed descending colon; src: UC Inflamed descending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282979 = Value for GSM282979: 5278 UC Inflamed descending colon; src: UC Inflamed descending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282998 = Value for GSM282998: 11410 UC Inflamed descending colon; src: UC Inflamed descending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM283013 = Value for GSM283013: 11429 UC Inflamed descending colon; src: UC Inflamed descending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM283017 = Value for GSM283017: 11434 UC Inflamed descending colon; src: UC Inflamed descending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM283018 = Value for GSM283018: 11436 UC Inflamed descending colon; src: UC Inflamed descending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM283025 = Value for GSM283025: 12463 UC Inflamed descending colon; src: UC Inflamed descending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM283028 = Value for GSM283028: 12466 UC Inflamed descending colon; src: UC Inflamed descending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM283032 = Value for GSM283032: 12470 UC Inflamed descending colon; src: UC Inflamed descending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM283037 = Value for GSM283037: 12475 UC Inflamed descending colon; src: UC Inflamed descending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM283040 = Value for GSM283040: 12478 UC Inflamed descending colon; src: UC Inflamed descending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM283042 = Value for GSM283042: 12480 UC Inflamed descending colon; src: UC Inflamed descending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM283045 = Value for GSM283045: 12483 UC Inflamed descending colon; src: UC Inflamed descending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM283048 = Value for GSM283048: 12486 UC Inflamed descending colon; src: UC Inflamed descending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM283052 = Value for GSM283052: 12490 UC Inflamed descending colon; src: UC Inflamed descending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM283054 = Value for GSM283054: 12492 UC Inflamed descending colon; src: UC Inflamed descending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282980 = Value for GSM282980: 5279 UC Inflamed ascending colon; src: UC Inflamed ascending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282982 = Value for GSM282982: 5281 UC Inflamed ascending colon; src: UC Inflamed ascending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282984 = Value for GSM282984: 5283 UC Inflamed ascending colon; src: UC Inflamed ascending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282986 = Value for GSM282986: 5285 UC Inflamed ascending colon; src: UC Inflamed ascending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM282997 = Value for GSM282997: 11409 UC Inflamed ascending colon; src: UC Inflamed ascending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM283012 = Value for GSM283012: 11428 UC Inflamed ascending colon; src: UC Inflamed ascending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM283027 = Value for GSM283027: 12465 UC Inflamed ascending colon; src: UC Inflamed ascending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM283031 = Value for GSM283031: 12469 UC Inflamed ascending colon; src: UC Inflamed ascending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM283039 = Value for GSM283039: 12477 UC Inflamed ascending colon; src: UC Inflamed ascending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM283044 = Value for GSM283044: 12482 UC Inflamed ascending colon; src: UC Inflamed ascending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
#GSM283047 = Value for GSM283047: 12485 UC Inflamed ascending colon; src: UC Inflamed ascending colon; src: Stratagene Universal Human Reference RNA (Catalog #740000)
"""

In [6]:
clean = lambda string: string.replace(':', '').split()
predicate = lambda word: word.startswith('GSM')

healthy = set(filter(predicate, clean(healthy_raw)))
unhealthy = set(filter(predicate, clean(unhealthy_raw)))

In [7]:
len(healthy),len(unhealthy)

(69, 63)

In [9]:
healthy = list(healthy)[:16]
unhealthy = list(unhealthy)[:16]

pickle.dump({"helthy":healthy,
             "unhealthy":unhealthy},
           open("colon_cancer_subset","wb"))

In [10]:
healthy = set(healthy)
unhealthy = set(unhealthy)

In [11]:
sample_columns = list(healthy | unhealthy)

In [12]:
cle = colon_cancer_dataset[["ID_REF","IDENTIFIER"]+sample_columns]

In [13]:
cle.dropna(inplace=True)

/home/ammar/virtualenv/bio/lib/python3.5/site-packages/ipykernel_launcher.py:1 SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [14]:
cle[cle.columns[:5]].head()

,ID_REF,IDENTIFIER,GSM282877,GSM282890,GSM282903
2,3,A_23_P80353,-0.131370,-0.118210,0.017929
3,4,APBA2,-0.333340,-0.150720,-0.117420
4,5,MAP3K6,-0.043199,0.258450,-0.035832
5,6,ZNF121,-0.485560,-0.307370,-0.561780
8,9,BX094364,-0.200700,-0.005776,0.002782


In [15]:
labels_ = [(h, 'healthy') for h in healthy] + [(d, 'unhealthy') for d in unhealthy]

In [17]:
X, y = parse_database(cle, labels=dict(labels_), n_jobs=-1)

[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed: 13.2min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed: 14.1min
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed: 14.2min
[Parallel(n_jobs=-1)]: Done  21 out of  32 | elapsed: 15.4min remaining:  8.1min
[Parallel(n_jobs=-1)]: Done  25 out of  32 | elapsed: 16.2min remaining:  4.5min
[Parallel(n_jobs=-1)]: Done  29 out of  32 | elapsed: 16.3min remaining:  1.7min


GSM282877 added with length 16990
GSM282890 added with length 16990
GSM283030 added with length 16990
GSM282915 added with length 16990
GSM283018 added with length 16990
GSM282857 added with length 16990
GSM282903 added with length 16990
GSM282896 added with length 16990
GSM283037 added with length 16990
GSM283033 added with length 16990
GSM282863 added with length 16990
GSM283040 added with length 16990
GSM282895 added with length 16990
GSM282982 added with length 16990
GSM282909 added with length 16990
GSM282913 added with length 16990
GSM282964 added with length 16990
GSM282921 added with length 16990
GSM282885 added with length 16990
GSM282961 added with length 16990
GSM283044 added with length 16990
GSM282969 added with length 16990
GSM283017 added with length 16990
GSM282945 added with length 16990
GSM282904 added with length 16990
GSM282898 added with length 16990
GSM282936 added with length 16990
GSM282886 added with length 16990
GSM282934 added with length 16990
GSM282972 adde

[Parallel(n_jobs=-1)]: Done  32 out of  32 | elapsed: 16.3min finished


In [18]:
# pickle.dump({"X":X,"y":y}, open("colon_cancer_Xy", "wb"))

In [3]:
coca = pickle.load(open("colon_cancer_Xy", "rb"))
X = coca["X"]
y = coca["y"]

## FVA

In [6]:
results = flux_variance_analysis(X, y)
pickle.dump(results, open("colon_cancer.results", "wb"))

../classes/genobolitics.py:72 UserWarning: model returned the empty string as reaction rule!
../classes/genobolitics.py:36 UserWarning: some operands are missing from logical expression!
../classes/genobolitics.py:56 UserWarning: could not evaluate boolean expression, objective-coeff is set to ZERO!
../classes/genobolitics.py:72 UserWarning: model returned the empty string as reaction rule!
../classes/genobolitics.py:56 UserWarning: could not evaluate boolean expression, objective-coeff is set to ZERO!
../classes/genobolitics.py:36 UserWarning: some operands are missing from logical expression!
../classes/genobolitics.py:72 UserWarning: model returned the empty string as reaction rule!
../classes/genobolitics.py:56 UserWarning: could not evaluate boolean expression, objective-coeff is set to ZERO!
../classes/genobolitics.py:36 UserWarning: some operands are missing from logical expression!
../classes/genobolitics.py:72 UserWarning: model returned the empty string as reaction rule!
../c

cobra/util/solver.py:403 UserWarning: solver status is 'infeasible'
../classes/genobolitics.py:72 UserWarning: model returned the empty string as reaction rule!
../classes/genobolitics.py:56 UserWarning: could not evaluate boolean expression, objective-coeff is set to ZERO!
../classes/genobolitics.py:36 UserWarning: some operands are missing from logical expression!
/usr/local/lib/python3.5/dist-packages/optlang/cplex_interface.py:459 DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
../classes/genobolitics.py:72 UserWarning: model returned the empty string as reaction rule!
../classes/genobolitics.py:56 UserWarning: could not evaluate boolean expression, objective-coeff is set to ZERO!
../classes/genobolitics.py:36 UserWarning: some operands are missing from logical expression!
/usr/local/lib/python3.5/dist-packages/optlang/cplex_interface.py:459 DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
../classes/genobolitics.py:72 UserWarnin

/usr/local/lib/python3.5/dist-packages/optlang/cplex_interface.py:459 DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
cobra/util/solver.py:403 UserWarning: solver status is 'infeasible'
